# TX00DQ05-3001 Exercises 2

Note that you don't have to use the functions / other code in the cells below. They are there just in case you need inspiration to get started.

In [1]:
import numpy as np
print(np.__version__)
import numpy.linalg as LA

1.16.2


## Exercise 1: Iterative policy evaluation. 

Calculate state-value function V for the gridworld of Sutton & Barto example 4.1. Policy is assumed to be random, ie. each of the four directions are equally likely. Movement that would result in leaving the grid (for example moving up in top row) will leave state unchanged (but action has been taken). Gamma (discount factor) is assumed to be = 1, ie. no discounting.

When norm of the difference between new V and the old one is less than eps, stop iteration.

Compare needed number of iterations between synchronous (sweep over all states, and update value function after the sweep) and asynchronous (use always the latest values) update of state-value function.

Note that numpy tensor assignment does not create a copy. You might want to use .copy() method to avoid sharing a reference to the same array.


    
 ## How to do exercise 1 policy evaluation
    
 




I'm going to borrow a phrase from teacher's playbook. 
#### Isn't it so that...


Original formula was $$ V_{k+1} =  \sum \limits_{a} { \pi(a|s) } * \sum \limits_{s',r} { [p( s', r | s, a )* (r + \gamma * V(s') ) ]} $$

We should be able to make some nice simplification into this monster formula as follows

* substitute practical values into formula
* use algebraic simplification

input practical values because we know the following values for gridworld
in these variables are in fact constants in the gridworld random walk policy...


Especially because it was random walk policy, then isn't it so that the probability p(s', r | s,a ) must be  0.25, because it was random walk, and you could end up in any of the four possible newStates with equal probability
because it was said in the slides as follows:

"Probability of ending up in
state s’ when taking action a
in state s. When multiplied
with the value of state s’ and
summed up, this gives the
expected value. "

Also isn't it so that the sum_for_all_actions( pi(a|s) ) = 1, by definition...

Algebraically, isn't it also so... that if you have calculation like (0.33 + 0.32 + 0.17 +0.18) * (w+x+y+z) = 1*(w+x+y+z)

$$ \gamma = 1 \\ 
reward = r= -1  \\  
p( s', r | s, a ) = 0.25  \\ 
\sum \limits_{a} { \pi(a|s) } = 0.25 + 0.25 + 0.25 + 0.25 = 1 $$

So that the simplifications can be made into a simpler formula such as follows:


$$ V_{k+1} = 0,25 * \sum \limits_{s'} { (  -1 + V(s') ) } $$

* so that you have to substitute the constant values into the original formula
* so that you move the p(s',r|s,a) outside the sum sign, because it is constant factor and it will be multiplied it is allowed
* then you are left with the sum_for_all_newStates( -1 + V(s')  ) under the influence of the sum formula. Also it will be so, that the sum formula will run for 1,2,3,4 newStates always (up right down left), but some of those newStates might be the same as oldState because you bump into the wall
* the minus one, is simply the constant value reward always -1
* for the cases in the summation, where you will iterate into the wall, and get bounced back from the newState (s') then it will simply be that way that the newState will be the same as the originalState (s)






In [2]:
rows_count = 4
columns_count = 4
V = np.zeros((rows_count, columns_count))
V_new = np.zeros((rows_count, columns_count))
stepcost = -1 #probably not needed
directions = ['up', 'right', 'down', 'left'] #probably not needed
maxiters = 1000000
eps = 0.0000001
k = 0 # "memory counter" of iterations inside the for loop, note that for loop i-variable is regular loop variable

def isTerminal(r,c, maxR, maxC):      #helper function to check if terminal state or regular state
    if r == 0 and c == 0: #im a bit too lazy to check otherwise the iteration boundaries        
        return True       #so that this helper function is a quick way to exclude computations
    if r == maxR-1 and c == maxC-1:
        return True
    return False

def getValue(row, col, V):    #you only need this getter function to access the oldArrayV
    if row == -1: row =0   #in the algorithm
    elif row == 4: row = 3
    if col == -1: col = 0
    elif col == 4: col =3
        
    return V[row,col]

"""synchronous case"""
"""Iterative Policy Evaluation algorithm, for estimating V ~ v_pi
using randomWalk policy"""
#basically you choose maxiters = k iterations
#as it was described in the gridworld example K= iteration count
# if maxiters =0 => results in 0 board
# if maxiters =1 => results in -1 board, except terminal state
# if maxiters =2 => results in that changed board





for i in range(maxiters):
    delta = 0
    for row in range(4):    #for all states
        for column in range(4):
            
            if(isTerminal(row, column, rows_count, columns_count)): # in case of terminal state
                newVal = 0
            else:
                oldVal = getValue(row, column, V) # in case of regular state
                newVal = 0.25 * (-4 + \
                                getValue(row, column-1, V) + \
                                getValue(row+1, column, V) + \
                                getValue(row, column+1, V) + \
                                getValue(row-1, column, V)  )
                V_new[row, column]= newVal   # update the values into the newArrayV
                delta = max(delta, abs(oldVal - newVal))
    V = V_new.copy() #update old array into previouosly calculated newVersion
    k+=1 # use k-variable as the "memory counter" so that you can keep the iteration count in memory after for loop
    if delta < eps: #epsilon convergence condition for early break
        break
    


print("iteration", k)
with np.printoptions(precision=3):
    print(V)
    
print("iteration", k)
with np.printoptions(precision=3):
    print(V_new)

iteration 300
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]
iteration 300
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]


In [3]:
"""asynchronous case"""
"""Iterative Policy Evaluation algorithm, for estimating V ~ v_pi
using randomWalk policy

NOTE::!!! USE THE OLDARRAY V ONLY!!!"""

V = np.zeros((rows_count, columns_count))    #reset the arrays
V_new = np.zeros((rows_count, columns_count))



#basically you choose maxiters = k iterations
#as it was described in the gridworld example K= iteration count
# if maxiters =0 => results in 0 board
# if maxiters =1 => results in -1 board, except terminal state
# if maxiters =2 => results in that changed board
k = 0 # "memory counter" of iterations inside the for loop, note that for loop i-variable is regular loop variable




for i in range(maxiters):
    delta = 0
    for row in range(4):    #for all states
        for column in range(4):
            
            if(isTerminal(row, column, rows_count, columns_count)): # in case of terminal state
                newVal = 0
            else:
                oldVal = getValue(row, column, V) # in case of regular state
                newVal = 0.25 * (-4 + \
                                getValue(row, column-1, V) + \
                                getValue(row+1, column, V) + \
                                getValue(row, column+1, V) + \
                                getValue(row-1, column, V)  )
                V[row, column]= newVal   # update the values into the oldArrayV, we are only using that old array only!!!
                delta = max(delta, abs(oldVal - newVal))
    #V = V_new.copy() #update old array into previouosly calculated newVersion
    k+=1 # use k-variable as the "memory counter" so that you can keep the iteration count in memory after for loop
    if delta < eps: #epsilon convergence condition for early break
        break
    


print("iteration", k)
with np.printoptions(precision=3):
    print(V)



iteration 193
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]


## Comparing asynch vs synch policy evaluation algorithms

* it appears true that in this case, at least, the asynch version was converging more rapidly, quite a bit faster

## Exercise 2: Greedy policy. 

Based on the state-value function computed in exercise 1, print out deterministic greedy policy function. Is the policy generated also optimal one?

## How to do exercise 2 greedy policy

* based on my reading of the book, the policyEval+Greedy should provide us the optimal policy assuming you let the algorithm run its own course and and when it's done

* if you only make 1 iteration of the full algorithm it's not guaranteed to converge in that short amount of iterations


## how to print the policy (probably use a table like the gridworld example shows)

* I think the good way to represent policy function
   (if that's  even a real thing, and not fake news)
   would be to have matrix of int, which are basically the direction arrows
   to follow in the policy. This is like the arrows that are in the gridworld
   policy map.


* int in matrix should initialy be random directional == 15 
  then, you could represent that with a start or something

* then you print them you can print nice unicode arrow signs, with regular directions (actions) 

* Another choice might be simply numpy matrix of unicode chars, then you coiuld just store the arrows inside or some char

In [4]:
"""also collect the old State Value fucnt into safe storage actually, 
so we can later use deterministic greedy policy. I think we should prepare
to do the greedy optimization"""

stateValFunc = V.copy()




In [5]:
"""here are some helper function definitions, which
should help in the formulation of the policyEvaluate + greedy so we dont get bloated code
at least too much"""

def asynchPolEval(maxiters, rows, cols, eps, V, pol, rew):
    V = V.copy() ## for simplicity sakes, work with local copy of arrayV, and return arrayV afterwards
    Pol = pol.copy()
    Rew = rew[:]
    
    for i in range(maxiters): ## iterate policy evaluation
        delta = 0
        
        for r in range(rows):    ## for all states
            for c in range(cols):
            
                if(isTerminal(r, c, rows, cols)): ## in case of terminal state
                    continue
                else:
                    oldVal = getValue(r, c, V) ## in case of regular state
                    if Pol[r,c] == 'A':        ## if we still had the random walk policy
                        newVal = 0.25 * (sum(Rew) + getValue(r, c-1, V) + \
                                         getValue(r+1, c, V) + \
                                         getValue(r, c+1, V) + \
                                         getValue(r-1, c, V)  )
                    else: ## we have deterministic policy instead                    
                        curPol = Pol[r,c]
                        if curPol == "U": probs = [1,0,0,0] ## up, right, down left
                        elif curPol == "R":probs = [0,1,0,0] ## up, right, down left
                        elif curPol == "D":probs = [0,0,1,0] ## up, right, down left
                        elif curPol == "L":probs = [0,0,0,1] ## up, right, down left
                        

                        newVal = (probs[0] * (  Rew[0] + getValue(r-1, c, V) )  ) + \
                                (probs[1] * (  Rew[1] + getValue(r, c+1, V) )  ) + \
                                (probs[2] * (  Rew[2] + getValue(r+1, c, V) )  ) + \
                                (probs[3] * (  Rew[3] + getValue(r, c-1, V) )  ) 
                        
                    V[row, column]= newVal   ## update the values into the oldArrayV, we are only using that old array only!!!
                    delta = max(delta, abs(oldVal - newVal))
        if delta < eps: ## epsilon convergence condition for early break from iterate policy evaluation
            return V
            
            
            
def synchPolEval(maxiters, rows, cols, eps, V, V_new, pol, rew):
    V = V.copy() ## for simplicity sakes, work with local copy of arrayV, and return arrayV afterwards
    V_new = V_new.copy()
    Pol = pol.copy()
    Rew = rew[:]
    
    
    for i in range(maxiters):
        delta = 0
        for r in range(rows):    #for all states
            for c in range(cols):
                if(isTerminal(r, c, rows, cols)): # in case of terminal state
                    continue
                else:
                    oldVal = getValue(r, c, V) # in case of regular state
                    if Pol[r,c] == 'A': ## if we still had the random walk policy
                        newVal = 0.25 * (sum(Rew) + getValue(r, c-1, V) + \
                                         getValue(r+1, c, V) + \
                                         getValue(r, c+1, V) + \
                                         getValue(r-1, c, V)  )                    
                    else: ## we have deterministic policy instead                    
                        curPol = Pol[r,c]
                        if curPol == "U": probs = [1,0,0,0] ## up, right, down left
                        elif curPol == "R":probs = [0,1,0,0] ## up, right, down left
                        elif curPol == "D":probs = [0,0,1,0] ## up, right, down left
                        elif curPol == "L":probs = [0,0,0,1] ## up, right, down left
                        
                        newVal = (probs[0] * (  Rew[0] + getValue(r-1, c, V) )  ) + \
                                (probs[1] * (  Rew[1] + getValue(r, c+1, V) )  ) + \
                                (probs[2] * (  Rew[2] + getValue(r+1, c, V) )  ) + \
                                (probs[3] * (  Rew[3] + getValue(r, c-1, V) )  )  
                    
                    
                    V_new[r, c]= newVal   # update the values into the newArrayV
                    delta = max(delta, abs(oldVal - newVal))
        V = (V_new.copy()) #update old array into previouosly calculated newVersion
        if delta < eps: #epsilon convergence condition for early break
            return V

            


In [6]:
"""initial values for actions are A for any, which means all directions 
are the action in that state, in other words, it means exactly that we have random walk policy at start

"""


policies = np.array([ ['T','A','A','A'], 
                     ['A','A','A','A'],
                     ['A','A','A','A'],
                     ['A','A','A','T'] ])


print(policies)
print('A stands for any direction')
print('R is for right, D stands for down etc...')
print('T is for terminalNode, no action needed')



print("im afraid that I didnt find a good way to include partial actions into single cell \n \
such as  'up-right', because theorereically isnt it so... that  \n \
you will have 16 different combos for those actions in a single cell in the policies table \n \
this will be so, because you have basically 4bit binary number which is the possible actions in \n \
a certain state, but it can be any one of them (such that e.g. 0 == terminal node, \n \
and 14 == up-right-down, 15 == up-right-down-left, 12 == up-right )")


def getAction(row, col, policies):    #you only need this getter function to access 
    if row == -1: row =0   #policies array
    elif row == 4: row = 3
    if col == -1: col = 0
    elif col == 4: col =3
        
    return policies[row,col]



[['T' 'A' 'A' 'A']
 ['A' 'A' 'A' 'A']
 ['A' 'A' 'A' 'A']
 ['A' 'A' 'A' 'T']]
A stands for any direction
R is for right, D stands for down etc...
T is for terminalNode, no action needed
im afraid that I didnt find a good way to include partial actions into single cell 
 such as  'up-right', because theorereically isnt it so... that  
 you will have 16 different combos for those actions in a single cell in the policies table 
 this will be so, because you have basically 4bit binary number which is the possible actions in 
 a certain state, but it can be any one of them (such that e.g. 0 == terminal node, 
 and 14 == up-right-down, 15 == up-right-down-left, 12 == up-right )


In [7]:
"""make exactly one iteration of policyEvaluate + GreedyOpt algorithm"""



V0 = stateValFunc.copy()    ## initialize to correct initial state val funct,                            
V1 = stateValFunc.copy()  ## based on the randomwalk policy state val function

actionStringDict = {0:'U', 1:'R', 2:'D', 3:'L'} ##check what values to put into new policy, to replace old ones
rewards = [-1, -1, -1, -1]


with np.printoptions(precision=3):
    print("original Value function was \n",V0)


"""greedy policy"""
keepOpt = True

k = 0 ## memory counter

## only make one iteration with the policyEval+GreedyOpt algorithm, not guaranteed to be the best with 
## iterations count being this small
## However, I think in this case it might be enough to converge quite immediately
## also, if you change the main outer while loop condition to be while(keepOpt), then it 
## should give optimal policy in end


while k<1:
    k += 1 
    V0 = synchPolEval(maxiters, 4, 4, eps, V0, V1, policies, rewards)
    V1 = V0.copy() ## originally synchPolEval had twin return values, but I changed it somewhere...
    policyStable = True ## so that we have to remember to update V1 also...
    
    for r in range(4):
        for c in range(4): #for all states
            
            oldAct = policies[r,c] ## handle terminal node, in policies, nothing needs to be done, in first terminal,
            if oldAct == 'T':      ##  and last terminal must end
                continue
                
            else:    ##regular node    
                tempList = [getValue(r-1, c, V0) + rewards[0], 
                             getValue(r, c+1, V0) + rewards[1], 
                             getValue(r+1, c, V0) + rewards[2], 
                             getValue(r, c-1, V0) + rewards[3]]  
          
    ## we already have list of rewards, list of newStates, for each action possible
    ## I think that we can basically disregard the probability portion of the formula if we have deterministic greedy
    ## because that p(s',r|s,a) will be either zero or one, depending on what the newState s' will be
    ## such that... it will be physically impossible to have action == right, but the newState s' will be the upper state
    ## the corresponding action will always make corresponding newState, and nothing else but that...
    
    ## so... basically we just compute list of values for [up,right,down,left] directions, and
    ## each value will be ==  (r + V(s'))
    ## from list, find maxVal and get that index
    ## use that index to search into actionStringDict, which contains string values "u", "r", "d", "l"
    ## update policy
                
                tempMax = max(tempList)    ##get best value from newStates
                for n in range(len(tempList)): ## find what index it was
                    if tempMax == tempList[n]:
                        maxInd = n
                        break
                        
                maxChar = actionStringDict[maxInd] ##use the index to access into correct string from the dict
                newAct = maxChar
                policies[r, c] = newAct    
                     
                if oldAct != newAct:
                    policyStable = False
    if policyStable:
        keepOpt = False
                




print("iterations done, k == ",k)

with np.printoptions(precision=3):
    print("ending value function was \n",V0)
    
with np.printoptions(precision=3):
    print("policy was \n",policies)
    
print('')
print('as you can see, I only bothered with the case of single decision actions')
print('the book describes some of those actions as being "up or right", but this policies table with \n \
deterministic decisions should also be a correct optimal policy, but it contains only the single deterministic choices \n \
(which are optimal, but not necessarily complete as I just described)')

print('')



original Value function was 
 [[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]
iterations done, k ==  1
ending value function was 
 [[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]
policy was 
 [['T' 'L' 'L' 'L']
 ['U' 'L' 'L' 'D']
 ['U' 'U' 'D' 'D']
 ['U' 'R' 'R' 'T']]

as you can see, I only bothered with the case of single decision actions
the book describes some of those actions as being "coin  toss between up and right", but this policies table with 
 deterministic decisions should also be a correct optimal policy, but it contains only the single deterministic choices 
 (which are optimal, but not necessarily complete as I just described)



## in conclusion, by eyeballing this solution for a policy (policyTable)

* it is an optimal solution, judging by eyeball and loking at my own policy table and the book example policy table
* it is not necessarily the unique  solution(???) 
* I left out the partial choice actions from the policy table
  such as "up-right" and "down-left"

## Exercise 3: Value function and policy in modified gridworld.

Change the definition of the exercise 1 gridworld by assigning a cost of -8 to movement in "up" direction. Compute the value function and greedy policy based on the value function. Is the greedy policy optimal?

## How to solve ex3 greedyOpt, basic gridworld, but modified rewards

* I had to do a fair bit of re-coding for my policy evaluation algorithm, but it appears I managed to do that re-coding correctly in the asynchPolEval, and synchPolEval functions


* re-coding had to be done, because I had just hardcoded the policy evaluation to only use the randomwalk policy and only use the hardcoded rewards, and the randomwalk probability of choosing action (0.25 equiprobable).


* Sutton and Barto pseudocode was worse than useless, because it was incredibly misleading pseudocode, it would have been much better to write that pseudocode for algorithms in much smarter way of notation usage... Heck, even English language pseudocode would have been much more understandable!!!


* the policy in the end  should be optimal policy, because Sutton and Barto claimed it will converge to the optimal policy eventually, assuming algorithm was coded correctly

* we will do this exercise 3 properly such that we allow the algorithm to run its own course until it ends in optimal policy (when compared to exercise 2, we will not only iterate once, but allow it to iterate how many times it needs)



In [8]:
       
        ## reset for new attemp at ex3
print('reseting V0, V1 and policies, and initalize rewards \n')

policies = np.array([ ['T','A','A','A'], 
                     ['A','A','A','A'],
                     ['A','A','A','A'],
                     ['A','A','A','T'] ])
print(policies)
            
print('')
V0 = np.zeros((rows_count, columns_count))    #reset the arrays
V1 = np.zeros((rows_count, columns_count))


with np.printoptions(precision=3):
    print('V0',V0)
print('')
    
with np.printoptions(precision=3):
    print('V1',V1)
print('')

rewards = [-8, -1, -1, -1] ## must be in order up,right,down,left
print('rewards \n',rewards)
k = 0
print('k == ',k)

reseting V0, V1 and policies, and initalize rewards 

[['T' 'A' 'A' 'A']
 ['A' 'A' 'A' 'A']
 ['A' 'A' 'A' 'A']
 ['A' 'A' 'A' 'T']]

V0 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

V1 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

rewards 
 [-8, -1, -1, -1]
k ==  0


In [9]:
"""greedy policy"""
keepOpt = True

while keepOpt:
    k += 1 ## memory counter
    V0 = synchPolEval(maxiters, 4, 4, eps, V0, V1, policies, rewards)
    V1 = V0.copy() ## when this line was missing, it was subtle bug, because V1 would not have updated 
                   ## together with V0. Originally I had used twin return values in the function but I changed it somewhere
    policyStable = True
    
    for r in range(4):
        for c in range(4): #for all states
            
            oldAct = policies[r,c] ## handle terminal node, in policies, nothing needs to be done, in first terminal,
            if oldAct == 'T':      ##  and last terminal must end
                continue
                
            else:    ##regular node    
                tempList = [getValue(r-1, c, V0) + rewards[0], 
                             getValue(r, c+1, V0) + rewards[1], 
                             getValue(r+1, c, V0) + rewards[2], 
                             getValue(r, c-1, V0) + rewards[3]]  
          
    ## we already have list of rewards, list of newStates, for each action possible
    ## I think that we can basically disregard the probability portion of the formula if we have deterministic greedy
    ## because that p(s',r|s,a) will be either zero or one, depending on what the newState s' will be
    ## such that... it will be physically impossible to have action == right, but the newState s' will be the upper state
    ## the corresponding action will always make corresponding newState, and nothing else but that...
    
    ## so... basically we just compute list of values for [up,right,down,left] directions, and
    ## each value will be ==  (r + V(s'))
    ## from list, find maxVal and get that index
    ## use that index to search into actionStringDict, which contains string values "u", "r", "d", "l"
    ## update policy
                
                tempMax = max(tempList)    ##get best value from newStates
                for n in range(len(tempList)): ## find what index it was
                    if tempMax == tempList[n]:
                        maxInd = n
                        break
                        
                maxChar = actionStringDict[maxInd] ##use the index to access into correct string from the dict
                newAct = maxChar
                policies[r, c] = newAct    
                     
                if oldAct != newAct:
                    policyStable = False
    if policyStable:
        keepOpt = False
                




print("iterations done, k == ",k)

with np.printoptions(precision=3):
    print("ending value function was \n",V0)
    
with np.printoptions(precision=3):
    print("policy was \n",policies)
    
print('')
print('as you can see, I only bothered with the case of single decision actions')
print('the book describes some of those actions as being "up or right", but this policies table with \n \
deterministic decisions should also be a correct optimal policy, but it contains only the single deterministic choices \n \
(which are optimal, but not necessarily complete as I just described)')

print('')

iterations done, k ==  4
ending value function was 
 [[ 0. -1. -2. -3.]
 [-5. -4. -3. -2.]
 [-4. -3. -2. -1.]
 [-3. -2. -1.  0.]]
policy was 
 [['T' 'L' 'L' 'D']
 ['R' 'R' 'R' 'D']
 ['R' 'R' 'R' 'D']
 ['R' 'R' 'R' 'T']]

as you can see, I only bothered with the case of single decision actions
the book describes some of those actions as being "coin  toss between up and right", but this policies table with 
 deterministic decisions should also be a correct optimal policy, but it contains only the single deterministic choices 
 (which are optimal, but not necessarily complete as I just described)



## The results were such that, IT WAS INDEED OPTIMAL!

* because I apparently coded algorithm correctly??!!! YAY! give me a cookie for reward! (or at least  points for the lab...)


* We can see immediately from looking at the board  the following fact: 
  so we can see, such that if the reward for up == -8, then because the board size is only 4x4
  then by definition we must have such that we NEVER choose up


* also, looking at the policiesTable with eyeballs, it looks like it is an optimal solution, but not necessarily the unique one


* from the first row, you should move to the upper left corner basically. But on first row last column you can go left or down


* from the other rows, except the ones near the wall itself, it should be such that you can move down or right, but avoiding hitting the wall. Bascially from the other rows except the first one, you should keep sliding towards the bottom right corner as fast as possible with right or down moves



## Extra exercise: Policy iteration

Implement policy iteration, ie. create a policy with the help of the value function from previous policy and iterate until policy is stable.

## How to do exercise Extra

* I will just initialize randomwalk policy
* I will use reward -1 for all except down = -8
* I just noticed that in the exercises for the book (page. 82), it said that Sutton and Barto deliberately 
   coded a bug into their own pseudocode for policy iteration algorithm! The exercise said that
   sometimes the pseudocode possible will get stuck between two equally good policies
   
* I didnt fix that bug yet, but usuallt it seems like if you start with randomwalk policy, then it
  doesnt usually get stuck into forever loop
  
* it sounds like that bug could be inside the optimizer portion of the algorithm, because the policy
    evaluation portion of algorithm was done with for loop so it is guaranteed to not get blocked 
    in forever loop


* two possible fixes that I could think of might be  to have the optimizer also in forloop,

* or alternatively the bug could be that there are two numerically equal actions, so that
  they are  equal value... but the optimizer might keep bouncing between those... 

In [10]:
"""reset variables for policyEval + greedyOpt"""

       
        ## reset for new attemp at ex3
print('reseting V0, V1 and policies, and initalize rewards \n')

policies = np.array([ ['T','A','A','A'], 
                     ['A','A','A','A'],
                     ['A','A','A','A'],
                     ['A','A','A','T'] ])
print(policies)
            
print('')
V0 = np.zeros((rows_count, columns_count))    #reset the arrays
V1 = np.zeros((rows_count, columns_count))


with np.printoptions(precision=3):
    print('V0',V0)
print('')
    
with np.printoptions(precision=3):
    print('V1',V1)
print('')

rewards = [-1, -1, -8, -1] ## must be in order up,right,down,left
print('rewards \n',rewards)
k = 0
print('k == ',k)


reseting V0, V1 and policies, and initalize rewards 

[['T' 'A' 'A' 'A']
 ['A' 'A' 'A' 'A']
 ['A' 'A' 'A' 'A']
 ['A' 'A' 'A' 'T']]

V0 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

V1 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

rewards 
 [-1, -1, -8, -1]
k ==  0


In [11]:
"""greedy policy"""
keepOpt = True

while keepOpt:
    k += 1 ## memory counter
    V0 = synchPolEval(maxiters, 4, 4, eps, V0, V1, policies, rewards)
    V1 = V0.copy() ## when this line was missing, it was subtle bug, because V1 would not have updated 
                   ## together with V0. Originally I had used twin return values in the function but I changed it somewhere
    policyStable = True
    
    for r in range(4):
        for c in range(4): #for all states
            
            oldAct = policies[r,c] ## handle terminal node, in policies, nothing needs to be done, in first terminal,
            if oldAct == 'T':      ##  and last terminal must end
                continue
                
            else:    ##regular node    
                tempList = [getValue(r-1, c, V0) + rewards[0], 
                             getValue(r, c+1, V0) + rewards[1], 
                             getValue(r+1, c, V0) + rewards[2], 
                             getValue(r, c-1, V0) + rewards[3]]  
          
    ## we already have list of rewards, list of newStates, for each action possible
    ## I think that we can basically disregard the probability portion of the formula if we have deterministic greedy
    ## because that p(s',r|s,a) will be either zero or one, depending on what the newState s' will be
    ## such that... it will be physically impossible to have action == right, but the newState s' will be the upper state
    ## the corresponding action will always make corresponding newState, and nothing else but that...
    
    ## so... basically we just compute list of values for [up,right,down,left] directions, and
    ## each value will be ==  (r + V(s'))
    ## from list, find maxVal and get that index
    ## use that index to search into actionStringDict, which contains string values "u", "r", "d", "l"
    ## update policy
                
                tempMax = max(tempList)    ##get best value from newStates
                for n in range(len(tempList)): ## find what index it was
                    if tempMax == tempList[n]:
                        maxInd = n
                        break
                        
                maxChar = actionStringDict[maxInd] ##use the index to access into correct string from the dict
                newAct = maxChar
                policies[r, c] = newAct    
                     
                if oldAct != newAct:
                    policyStable = False
    if policyStable:
        keepOpt = False
                




print("iterations done, k == ",k)

with np.printoptions(precision=3):
    print("ending value function was \n",V0)
    
with np.printoptions(precision=3):
    print("policy was \n",policies)
    
print('')
print('as you can see, I only bothered with the case of single decision actions')
print('the book describes some of those actions as being "coin  toss between up and right", but this policies table with \n \
deterministic decisions should also be a correct optimal policy, but it contains only the single deterministic choices \n \
(which are optimal, but not necessarily complete as I just described)')

print('')

iterations done, k ==  4
ending value function was 
 [[ 0. -1. -2. -3.]
 [-1. -2. -3. -4.]
 [-2. -3. -4. -5.]
 [-3. -2. -1.  0.]]
policy was 
 [['T' 'L' 'L' 'L']
 ['U' 'U' 'U' 'U']
 ['U' 'U' 'U' 'U']
 ['U' 'R' 'R' 'T']]

as you can see, I only bothered with the case of single decision actions
the book describes some of those actions as being "coin  toss between up and right", but this policies table with 
 deterministic decisions should also be a correct optimal policy, but it contains only the single deterministic choices 
 (which are optimal, but not necessarily complete as I just described)



## results were such that

* it ended in optimal policy at this time
* algorithm managed to avoid going down, which was what we wanted
* algorithm went rightwards on the last row, when it was beneficial which is correct also
* on first row the choices were correct leftwards because it's the straightest line possible
* upwards and leftwards creep was to be expected otherwise

## Exercise 4: Value iteration

Solve the exercise 1 gridworld with value iteration algorithm. Solve also modified gridworld (cost of "up" movement = -4).

## how to solve exercise4 value iteration algorithm
* lets just test this basic algorithm first of all for the basic case
* if it works for the basic case, where all rewards = -1
* then it maybe works for other cases also ???!!!

In [12]:
"""reset variables for preparation for the value iteration algorithm"""

       
        ## reset for new attemp at ex3
print('reseting V0, V1 and policies, and initalize rewards \n')

policies = np.array([ ['T','A','A','A'], 
                     ['A','A','A','A'],
                     ['A','A','A','A'],
                     ['A','A','A','T'] ])
print(policies)
            
print('')
V0 = np.zeros((rows_count, columns_count))    #reset the arrays
V1 = np.zeros((rows_count, columns_count))


with np.printoptions(precision=3):
    print('V0',V0)
print('')
    
with np.printoptions(precision=3):
    print('V1',V1)
print('')

rewards = [-1, -1, -1, -1] ## must be in order up,right,down,left
print('rewards \n',rewards)
k = 0
print('k == ',k)

reseting V0, V1 and policies, and initalize rewards 

[['T' 'A' 'A' 'A']
 ['A' 'A' 'A' 'A']
 ['A' 'A' 'A' 'A']
 ['A' 'A' 'A' 'T']]

V0 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

V1 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

rewards 
 [-1, -1, -1, -1]
k ==  0


In [13]:
# value iteration

theta = 0.0001
maxiters = 10
#V = np.zeros((rows_count,columns_count)) ## Use arrayV0
directions = ["up", "right", "down", "left"]

"""value iteration algorithm"""

while k < maxiters:
    delta = 0
    k += 1 
    with np.printoptions(precision=3):
        print("k == ", k, ": value func == \n \n", V0)
    
    for r in range(rows_count):
        for c in range(columns_count):
            if not isTerminal(r,c, rows_count, columns_count):  ## for all regular states
            
                oldVal = V0[r,c]
                tempList = [getValue(r-1, c, V0) + rewards[0], 
                             getValue(r, c+1, V0) + rewards[1], 
                             getValue(r+1, c, V0) + rewards[2], 
                             getValue(r, c-1, V0) + rewards[3]]
                maxVal = max(tempList)
                for i in range(len(tempList)):
                    if maxVal == tempList[i]:
                        maxInd = i
                        break
                newVal = tempList[maxInd]
                V0[r,c] = newVal
                delta = ( max( delta, abs( oldVal - newVal ) ) )              

            else:
                continue ## this is probably bad coding practice, but I couldnt think of any other good solution
                         ## to prevent the iteration over the terminal states and break clause problems...
    if delta < theta:    ## actually it looks like the break checking delta < theta onlyy happens once per k iterations
        k = (maxiters + 100)  ## after each of the states (after the two-nested loop) has been iterated already, 
                              ## this appears to be good!
    
        
"""get the deterministic policy afterwards"""

for r in range(rows_count):
    for c in range(columns_count):
        if not isTerminal(r, c, rows_count, columns_count):
            tempList = [getValue(r-1, c, V0) + rewards[0], 
                        getValue(r, c+1, V0) + rewards[1], 
                        getValue(r+1, c, V0) + rewards[2], 
                        getValue(r, c-1, V0) + rewards[3]]
            tempMax = max(tempList)    ##get best value from newStates
            for n in range(len(tempList)): ## find what index it was
                if tempMax == tempList[n]:
                    maxInd = n
                    break
                        
            maxChar = actionStringDict[maxInd] ##use the index to access into correct string from the dict
            newAct = maxChar
            policies[r, c] = newAct

            
print("\n\n\n ending results were: \n")
with np.printoptions(precision=3):
    print("value function was \n", V0)
    
with np.printoptions(precision=3):
    print("policy was \n", policies)
                


k ==  1 : value func == 
 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
k ==  2 : value func == 
 
 [[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]
k ==  3 : value func == 
 
 [[ 0. -1. -2. -2.]
 [-1. -2. -2. -2.]
 [-2. -2. -2. -1.]
 [-2. -2. -1.  0.]]
k ==  4 : value func == 
 
 [[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



 ending results were: 

value function was 
 [[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
policy was 
 [['T' 'L' 'L' 'D']
 ['U' 'U' 'U' 'D']
 ['U' 'U' 'R' 'D']
 ['U' 'R' 'R' 'T']]


## looks like the value iteration algorithm worked at least for regular case
* regular case with rewards all of them == -1
* ending policy was an optimal one at the end
* this algorithm was very confusing once again, judging from the book chapter, and book examples (in fact there wasnt even a practical example at all, I think)

## let's check the modified case with value iteration

* adjust the up reward to be = -4
* other rewards = -1

* hope for the best, fingers crossed!!!!



In [14]:
"""reset variables for preparation for the value iteration algorithm"""

       
        ## reset for new attemp at ex3
print('reseting V0, V1 and policies, and initalize rewards \n')

policies = np.array([ ['T','A','A','A'], 
                     ['A','A','A','A'],
                     ['A','A','A','A'],
                     ['A','A','A','T'] ])
print(policies)
            
print('')
V0 = np.zeros((rows_count, columns_count))    #reset the arrays
V1 = np.zeros((rows_count, columns_count))


with np.printoptions(precision=3):
    print('V0',V0)
print('')
    
with np.printoptions(precision=3):
    print('V1',V1)
print('')

rewards = [-4, -1, -1, -1] ## must be in order up,right,down,left
print('rewards \n',rewards)
k = 0
print('k == ',k)

reseting V0, V1 and policies, and initalize rewards 

[['T' 'A' 'A' 'A']
 ['A' 'A' 'A' 'A']
 ['A' 'A' 'A' 'A']
 ['A' 'A' 'A' 'T']]

V0 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

V1 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

rewards 
 [-4, -1, -1, -1]
k ==  0


In [15]:
# value iteration

theta = 0.0001
maxiters = 50
#V = np.zeros((rows_count,columns_count)) ## Use arrayV0
directions = ["up", "right", "down", "left"]

"""value iteration algorithm"""

while k < maxiters:
    delta = 0
    k += 1 
    with np.printoptions(precision=3):
        print("k == ", k, ": value func == \n \n", V0)
    
    for r in range(rows_count):
        for c in range(columns_count):
            if not isTerminal(r,c, rows_count, columns_count):  ## for all regular states
            
                oldVal = V0[r,c]
                tempList = [getValue(r-1, c, V0) + rewards[0], 
                             getValue(r, c+1, V0) + rewards[1], 
                             getValue(r+1, c, V0) + rewards[2], 
                             getValue(r, c-1, V0) + rewards[3]]
                maxVal = max(tempList)
                for i in range(len(tempList)):
                    if maxVal == tempList[i]:
                        maxInd = i
                        break
                newVal = tempList[maxInd]
                V0[r,c] = newVal
                delta = ( max( delta, abs( oldVal - newVal ) ) )

            else:
                continue
                                 
    if delta < theta:  ## it appears that the break check delta < theta happens only after each k iterations, this is good,
        k = (maxiters + 100) ## because it appears that we will only check after all states are already iterated over,
                              ## after the two-nested loop has been run.... good!!!
                               ## these loops are difficult to code in python because of there is only whitespace
                                ## to guide our thinking and loop nesting...
        
"""get the deterministic policy afterwards"""

for r in range(rows_count):
    for c in range(columns_count):  ##iterate over all the actions  in policyTable
        if not isTerminal(r, c, rows_count, columns_count): ## in order to find the deterministic policy
            tempList = [getValue(r-1, c, V0) + rewards[0], 
                        getValue(r, c+1, V0) + rewards[1], 
                        getValue(r+1, c, V0) + rewards[2], 
                        getValue(r, c-1, V0) + rewards[3]]
            tempMax = max(tempList)    ##get best value from newStates
            for n in range(len(tempList)): ## find what index it was
                if tempMax == tempList[n]:
                    maxInd = n
                    break
                        
            maxChar = actionStringDict[maxInd] ##use the index to access into correct string from the dict
            newAct = maxChar
            policies[r, c] = newAct
            
print("\n\n\n ending results were: \n")
with np.printoptions(precision=3):
    print("value function was \n", V0)
    
with np.printoptions(precision=3):
    print("policy was \n", policies)
                


k ==  1 : value func == 
 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
k ==  2 : value func == 
 
 [[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]
k ==  3 : value func == 
 
 [[ 0. -1. -2. -2.]
 [-2. -2. -2. -2.]
 [-2. -2. -2. -1.]
 [-2. -2. -1.  0.]]
k ==  4 : value func == 
 
 [[ 0. -1. -2. -3.]
 [-3. -3. -3. -2.]
 [-3. -3. -2. -1.]
 [-3. -2. -1.  0.]]
k ==  5 : value func == 
 
 [[ 0. -1. -2. -3.]
 [-4. -4. -3. -2.]
 [-4. -3. -2. -1.]
 [-3. -2. -1.  0.]]



 ending results were: 

value function was 
 [[ 0. -1. -2. -3.]
 [-4. -4. -3. -2.]
 [-4. -3. -2. -1.]
 [-3. -2. -1.  0.]]
policy was 
 [['T' 'L' 'L' 'D']
 ['U' 'R' 'R' 'D']
 ['R' 'R' 'R' 'D']
 ['R' 'R' 'R' 'T']]


## it appears that the value iteration algorithm produced reasonable results

* I had to debug this value iteration inside SPYDER IDE with python debugger just to be sure about the break check delta < theta just to be sure


* it appears that you check delta < theta once per k iterations, such that you first will have already iterated over all the states, in the two-nested loop


* both policies the regular case, as well as the case of up reward = -4 appear to be optimal


* value iteration algorithm was more difficult to understand than the dynamic programming case with the other algorithm, policyEval + GreedyOpt


* however it appears that the value iteration algorithm is much superior in efficiency in convergence speed!!!!